# Import Data

In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_excel('/content/drive/MyDrive/Colab Data/Individual_Assignment.xlsx')

# Phase One1: Data Preprocessing and Cleaning

## Handling Missing Values

Identify columns with missing values

In [ ]:
missing_values = df.isnull().sum()
columns_with_missing_values = missing_values[missing_values > 0].index.tolist()

print("Columns with missing values:")
print(missing_values[columns_with_missing_values])

Columns with missing values:
MinTemp           1485
MaxTemp           1261
Rainfall          3261
Evaporation      62790
Sunshine         69835
WindGustDir      10326
WindGustSpeed    10263
WindDir9am       10566
WindDir3pm        4228
WindSpeed9am      1767
WindSpeed3pm      3062
Humidity9am       2654
Humidity3pm       4507
Pressure9am      15065
Pressure3pm      15028
Cloud9am         55888
Cloud3pm         59358
Temp9am           1767
Temp3pm           3609
RainToday         3261
RainTomorrow      3267
dtype: int64


Handle the missing values

In [ ]:
threshold = 0.3 * len(df)
df = df.dropna(thresh=threshold, axis=1)

for column in df.columns:
    if df[column].isnull().any():
        if df[column].dtype in ['int64', 'float64']:
            df[column].fillna(df[column].mean(), inplace=True)
        else:
            df[column].fillna(df[column].mode()[0], inplace=True)

print("\nMissing values after handling:")
print(df.isnull().sum())

/tmp/ipython-input-3249810723.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(df[column].mean(), inplace=True)
/tmp/ipython-input-3249810723.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try 


Missing values after handling:
Date             0
Location         0
MinTemp          0
MaxTemp          0
Rainfall         0
Evaporation      0
Sunshine         0
WindGustDir      0
WindGustSpeed    0
WindDir9am       0
WindDir3pm       0
WindSpeed9am     0
WindSpeed3pm     0
Humidity9am      0
Humidity3pm      0
Pressure9am      0
Pressure3pm      0
Cloud9am         0
Cloud3pm         0
Temp9am          0
Temp3pm          0
RainToday        0
RainTomorrow     0
dtype: int64


## Handling Outliers

Identify Outliers Using IQR

In [ ]:
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns

print("Identifying outliers using IQR:")
for col in numerical_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    if not outliers.empty:
        print(f"Column '{col}' has potential outliers.")

Identifying outliers using IQR:
Column 'MinTemp' has potential outliers.
Column 'MaxTemp' has potential outliers.
Column 'Rainfall' has potential outliers.
Column 'Evaporation' has potential outliers.
Column 'Sunshine' has potential outliers.
Column 'WindGustSpeed' has potential outliers.
Column 'WindSpeed9am' has potential outliers.
Column 'WindSpeed3pm' has potential outliers.
Column 'Humidity9am' has potential outliers.
Column 'Pressure9am' has potential outliers.
Column 'Pressure3pm' has potential outliers.
Column 'Cloud3pm' has potential outliers.
Column 'Temp9am' has potential outliers.
Column 'Temp3pm' has potential outliers.


Handle the Outliers

In [ ]:
for col in numerical_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df[col] = df[col].clip(lower=lower_bound, upper=upper_bound)

## Dealing with Duplicate Records

In [ ]:
df.drop_duplicates(inplace=True)

## Data Transformation

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
import pandas as pd

categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

target_col = 'RainTomorrow'

if target_col in df.columns and df[target_col].dtype == 'object':
    label_encoder = LabelEncoder()
    df[target_col] = label_encoder.fit_transform(df[target_col])

if target_col in categorical_cols:
    categorical_cols.remove(target_col)
if target_col in numerical_cols:
    numerical_cols.remove(target_col)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore', drop='first', sparse_output=False), categorical_cols)
    ],
    remainder='drop'
)

X = df.drop(columns=[target_col])
y = df[target_col]

X_transformed = preprocessor.fit_transform(X)

X_transformed = pd.DataFrame(X_transformed, columns=preprocessor.get_feature_names_out())


# Phase 2: Feature Engineering

## Creating New Features

In [ ]:
df['TempRange'] = df['MaxTemp'] - df['MinTemp']

df['WindSpeed9am_sq'] = df['WindSpeed9am']**2
df['WindSpeed3pm_sq'] = df['WindSpeed3pm']**2

df['Humidity9am_Temp9am_Interaction'] = df['Humidity9am'] * df['Temp9am']
df['Humidity3pm_Temp3pm_Interaction'] = df['Humidity3pm'] * df['Temp3pm']

if not pd.api.types.is_datetime64_any_dtype(df['Date']):
    df['Date'] = pd.to_datetime(df['Date'])

df['DayOfWeek'] = df['Date'].dt.dayofweek
df['Month'] = df['Date'].dt.month

display(df.head())

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Temp3pm,RainToday,RainTomorrow,TempRange,WindSpeed9am_sq,WindSpeed3pm_sq,Humidity9am_Temp9am_Interaction,Humidity3pm_Temp3pm_Interaction,DayOfWeek,Month
0,2008-12-01,Albury,13.4,22.9,0.6,5.468232,7.611178,W,44.0,W,...,21.8,No,0,9.5,400.0,576.0,1199.9,479.6,0,12
1,2008-12-02,Albury,7.4,25.1,0.0,5.468232,7.611178,WNW,44.0,NNW,...,24.3,No,0,17.7,16.0,484.0,756.8,607.5,1,12
2,2008-12-03,Albury,12.9,25.7,0.0,5.468232,7.611178,WSW,46.0,W,...,23.2,No,0,12.8,361.0,676.0,798.0,696.0,2,12
3,2008-12-04,Albury,9.2,28.0,0.0,5.468232,7.611178,NE,24.0,SE,...,26.5,No,0,18.8,121.0,81.0,814.5,424.0,3,12
4,2008-12-05,Albury,17.5,32.3,1.0,5.468232,7.611178,W,41.0,ENE,...,29.7,No,0,14.8,49.0,400.0,1459.6,980.1,4,12


## Dimensionality Reduction

In [ ]:
from sklearn.decomposition import PCA

n_components = 10
pca = PCA(n_components=n_components)

X_pca = pca.fit_transform(X_transformed)

print(f"Original number of features: {X_transformed.shape[1]}")
print(f"Number of features after PCA: {X_pca.shape[1]}")
print(f"Explained variance ratio by {n_components} components:", pca.explained_variance_ratio_.sum())

Original number of features: 110
Number of features after PCA: 10
Explained variance ratio by 10 components: 0.7801281758576715


## Binning and Discretization

In [ ]:
df['Rainfall_Binned'] = pd.cut(df['Rainfall'], bins=3, labels=['Low', 'Medium', 'High'])

display(df[['Rainfall', 'Rainfall_Binned']].head())

,Rainfall,Rainfall_Binned
0,0.6,Low
1,0.0,Low
2,0.0,Low
3,0.0,Low
4,1.0,Medium


## Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler

numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns

scaler = StandardScaler()

df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

display(df.head())

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,RainToday,RainTomorrow,TempRange,WindSpeed9am_sq,WindSpeed3pm_sq,Humidity9am_Temp9am_Interaction,Humidity3pm_Temp3pm_Interaction,DayOfWeek,Month,Rainfall_Binned
0,2008-12-01,Albury,0.189440,-0.046005,-0.014673,0.217795,-0.224518,W,0.351845,W,...,No,-0.529764,-0.309399,0.454269,0.450643,0.192742,-1.335690,0,12,Low
1,2008-12-02,Albury,-0.753297,0.265277,-0.621711,0.217795,-0.224518,WNW,0.351845,NNW,...,No,-0.529764,1.347923,-0.865880,0.190973,-0.811375,-1.030985,1,12,Low
2,2008-12-03,Albury,0.110879,0.350172,-0.621711,0.217795,-0.224518,WSW,0.516118,W,...,No,-0.529764,0.357572,0.320192,0.732892,-0.718011,-0.820145,2,12,Low
3,2008-12-04,Albury,-0.470476,0.675603,-0.621711,0.217795,-0.224518,NE,-1.290879,SE,...,No,-0.529764,1.570247,-0.504901,-0.946491,-0.680620,-1.468150,3,12,Low
4,2008-12-05,Albury,0.833644,1.284019,0.390019,0.217795,-0.224518,W,0.105437,ENE,...,No,-0.529764,0.761797,-0.752429,-0.046116,0.781253,-0.143313,4,12,Medium
